## Key Topics Extraction and Contextual Sentiment of Hotel Reviews

## 1. Preprocess

In [3]:
import os
from os import listdir
import pandas as pd
#!pip install plotly
import plotly.graph_objs as go
df = pd.read_csv('hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv', header=0)
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'],
      dtype='object')

In [4]:
df.drop(columns=['dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'keys', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username', 'sourceURLs', 'websites'], inplace = True)
df = df.rename(index=str, columns={'reviews.rating':'rating', 'reviews.sourceURLs': 'reviews_sourceURLs', 'reviews.text':'reviews_text', 'reviews.title': 'reviews_title'})

In [14]:
df.head()

,id,city,country,latitude,longitude,name,postalCode,province,rating,reviews_sourceURLs,reviews_text,reviews_title
0,AWE2FvX5RxPSIh2RscTK,Goleta,US,34.44178,-119.81979,Best Western Plus South Coast Inn,93117,CA,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel
1,AVwcj_OhkufWRAb5wi9T,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel
2,AVwcj_OhkufWRAb5wi9T,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,3,https://www.tripadvisor.com/Hotel_Review-g3217...,"Parking was horrible, somebody ran into my ren...",Business
3,AVwcj_OhkufWRAb5wi9T,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,5,https://www.tripadvisor.com/Hotel_Review-g3217...,Not cheap but excellent location. Price is som...,Very good
4,AVwcj_OhkufWRAb5wi9T,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,2,https://www.tripadvisor.com/Hotel_Review-g3217...,If you get the room that they advertised on th...,Low chance to come back here


In [5]:
f= open("hotel_review.txt","w+")
for i in range(df.shape[0]):
    f.write(df.reviews_text[i])
    f.write("\n")
f.close()

In [6]:
with open("hotel_review.txt") as fp: 
    reviews = []
    Lines = fp.readlines() 
    for line in Lines[:5]: 
        reviews.append(line)

In [7]:
reviews

['This hotel was nice and quiet. Did not know, there was train track near by. But it was only few train passed during our stay. Best Western changed hotel classification. The Plus category are not the same as before.\n',
 "We stayed in the king suite with the separation between the bedroom and the living space. The sofa bed wasn't very good I had back discomfort by the day we left on our three night stay. The room is clean, and the king bed very comfortable. This hotel is located within walking distance to most places you will want to... More\n",
 "Parking was horrible, somebody ran into my rental car while staying there. I didn't get to try the breakfast, I was there for business so the restaurant opened to late for the business world to enjoy, I had to asked for coffee for my room, And the items in the vending machine were stale.\n",
 'Not cheap but excellent location. Price is somewhat standard for not hacing reservations. But room was nice and clean. They offer good continental bre

## 2. Amenity Extraction

In [24]:
import nltk.data
from os import listdir
from os.path import isfile, join
from nltk.util import bigrams 
from nltk.tokenize import TreebankWordTokenizer
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
treebank_tokenizer = TreebankWordTokenizer()
import numpy as np
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
#stop_words = stopwords.words('english')

In [25]:
#check if the element passed in is a float
def isfloat(value):
  try:
    float(value)
    #return true if it can be cast to a float
    return True
  except ValueError:
    return False

In [208]:
stop_words = ['in','of','at','a','the']
def process_text(text):
    #tokenzie the text as sentences
    punkt_sentences = sentence_tokenizer.tokenize(text)
    #tokenize as lists of sentence lists of words
    sentences_words = [treebank_tokenizer.tokenize(sentence) for sentence in punkt_sentences]
    #get a list of words without punctuations
    tokens = [word for sentence in sentences_words for word in sentence if word == 'A/C' or word.isalpha()]
    #lemmatization 
    tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
    #remove the stopwords
    tokens = [word for word in tokens if not word in stop_words]
    #remove numbers
    tokens = [word for word in tokens if isfloat(word == False)]
    #generate bigrams
    bigrams = nltk.bigrams(tokens)
    return tokens,list(bigrams)

### Train a sentiment classifer

In [134]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
 
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
negcutoff = int(len(negfeats)*3/4)
poscutoff = int(len(posfeats)*3/4)
 
#print(negcutoff)
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
#print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))
 
classifier = NaiveBayesClassifier.train(trainfeats)
#print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
#classifier.show_most_informative_features()

In [203]:
import pickle
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [204]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [214]:
def senti(bigram):
    return classifier.classify(word_feats(bigram.split())) == 'pos'
sent('good')

False

In [211]:
amenity_list = ['A/C','fan','parking','coffee','breakfast','pool','clean',
             'kitchen','internet','computer','recreation','exercise',
             'vending','locker','dryer','laundry']
def amenity_extraction_hotel(hotel):
    amenities = {}
    for text in df[df['name'] == hotel]['reviews_text']:
        tokens,bigrams = process_text(text)
        for word in tokens:
            for amenity in amenity_list:
                if word.lower() == amenity.lower():
                    if word not in amenities.keys():
                        amenities.update({word:0})
                    for bi in bigrams:
                        if bi[1].lower == word.lower:
                            #print(bi[0])
                            #print(senti(bi[0]))
                            if senti(bi[0]):
                                amenities[word] += 1
                            else:
                                amenities[word] -= 1               
    return amenities

In [212]:
#amenity_extraction_hotel(df['name'][4])

is clean
False
try breakfast
True
for coffee
True
item vending
False
and clean
True
continental breakfast
False
no A/C
False
have fan
False
near pool
True


{'clean': 0,
 'Parking': 0,
 'breakfast': 0,
 'coffee': 1,
 'vending': -1,
 'A/C': -1,
 'fan': -1,
 'pool': 1}

### Extract amenties for all the hotels

In [213]:
hotel_amenities = {}
for hotel in df['name'].unique():
    amenities = amenity_extraction_hotel(hotel)
    hotel_amenities.update({hotel:amenities})

is clean
False
try breakfast
True
for coffee
True
item vending
False
and clean
True
continental breakfast
False
no A/C
False
have fan
False
near pool
True
industrial fan
True
industrial fan
True
good breakfast
False
swimming pool
False
Nice breakfast
True
across parking
False
across parking
False
wa clean
True
The breakfast
True
wa clean
True
functional Breakfast
True
wa clean
True
clean Parking
True
very clean
True
The breakfast
True
wa clean
True
job Breakfast
True
free breakfast
True
family breakfast
True
cold breakfast
True
were clean
False
and breakfast
True
wa clean
True
reasonable Breakfast
False
hot breakfast
True
City Clean
True
Great Breakfast
True
very clean
True
The breakfast
True
A clean
True
The breakfast
True
amenity Parking
True
Broken exercise
True
updated exercise
True
basic breakfast
False
very clean
True
mentioning breakfast
True
really clean
True
Valet parking
True
your parking
True
ne breakfast
False
th breakfast
True
city Parking
False
continental breakfast
False

leave fan
True
leave fan
True
have fan
False
The breakfast
True
Since breakfast
True
and clean
True
up breakfast
False
enjoyed pool
True
u pool
True
wa clean
True
The pool
True
to clean
False
into breakfast
True
is fan
False
wa clean
True
about parking
False
an fan
True
wa clean
True
and coffee
True
full breakfast
False
The A/C
True
daily parking
True
The breakfast
True
enjoyed breakfast
True
wa pool
True
looked clean
False
The breakfast
True
making breakfast
True
what coffee
False
hot breakfast
True
The coffee
True
for clean
True
big clean
False
The breakfast
True
The breakfast
True
ventilation fan
True
and pool
True
hotel clean
True
The pool
True
from pool
True
complimentary breakfast
True
my breakfast
False
includes breakfast
False
The breakfast
True
offer laundry
True
fridge coffee
False
making breakfast
True
fresh coffee
True
daily breakfast
True
loved pool
True
The breakfast
True
and breakfast
True
loved pool
True
liked breakfast
True
nice pool
True
up fan
False
working Breakfast

VERY clean
True
ceiling Pool
False
sortof clean
True
nice breakfast
True
Very clean
True
great breakfast
True
good breakfast
False
so clean
False
very clean
True
and breakfast
True
exhaust fan
True
nice clean
True
clean breakfast
True
TV coffee
True
quiet clean
True
HOT breakfast
True
very clean
True
Country Kitchen
True
is clean
False
very clean
True
large parking
True
nice pool
True
pool breakfast
False
very clean
True
for breakfast
True
very clean
True
free breakfast
True
and clean
True
wa clean
True
for clean
True
and clean
True
I breakfast
True
hot breakfast
True
on breakfast
False
very clean
True
are breakfast
True
wa clean
True
indoor pool
True
wa clean
True
comfortable Breakfast
False
airport clean
True
needed Coffee
False
wa clean
True
next breakfast
True
and clean
True
Quarter Clean
True
enjoy breakfast
True
warm pool
True
and clean
True
is breakfast
False
hour Breakfast
False
have coffee
False
and coffee
True
wa clean
True
and clean
True
wa clean
True
The pool
True
were clea

are clean
True
clean Parking
True
hot breakfast
True
and pool
True
room Breakfast
True
free breakfast
True
perk breakfast
True
snack pool
False
pool exercise
False
secure parking
True
are clean
True
small coffee
True
wa clean
True
for breakfast
True
already clean
False
The breakfast
True
and clean
True
visit coffee
True
cup coffee
False
hot breakfast
True
accommodation Breakfast
True
nice pool
True
Very clean
True
nice pool
True
Very clean
True
suite breakfast
True
extremely clean
True
Complimentary breakfast
True
Very clean
True
amenity Pool
True
very clean
True
complimentary breakfast
True
shopping Clean
False
back Breakfast
True
Good coffee
True
area clean
True
very clean
True
Nice pool
True
customer Breakfast
True
were clean
False
wa clean
True
hotel parking
True
spacious clean
True
super clean
False
place Clean
True
wa clean
True
ago Clean
False
free parking
True
car parking
False
ample parking
True
very clean
True
very clean
True
and parking
True
park Parking
True
for breakfast
T

False
weekend breakfast
False
and clean
True
touch Breakfast
True
lap pool
False
Very clean
True
such coffee
True
super clean
False
free coffee
True
beautiful clean
True
wa clean
True
Louisville fan
True
suite kitchen
True
equipped kitchen
True
enjoyed breakfast
True
enjoyed breakfast
True
stocked kitchen
True
Included breakfast
True
wa clean
True
overall pool
True
hotel pool
True
swimming pool
False
for parking
True
wa clean
True
Has pool
True
outdoor pool
True
quality breakfast
False
were clean
False
u pool
True
away Pool
True
made breakfast
False
best Parking
True
and clean
True
be clean
False
for internet
True
great pool
True
were clean
False
and pool
True
and breakfast
True
that breakfast
True
our breakfast
True
The parking
True
hot breakfast
True
employee breakfast
False
super clean
False
great breakfast
True
wa clean
True
knowledgeable parking
True
night Parking
True
is parking
False
their parking
True
there Breakfast
False
egg Breakfast
True
eating breakfast
True
wa clean
True


...

is clean
False
sold Breakfast
True
quickly Parking
False
The breakfast
True
and pool
True
super clean
False
am clean
False
bed clean
False
clean A/C
True
into parking
True
room clean
True
channel Breakfast
False
helpful Pool
True
and clean
True
are clean
True
missed breakfast
False
good clean
False
facing pool
True
pool pool
False
but breakfast
True
people coffee
True
them Laundry
True
wa clean
True
The pool
True
scoured internet
True
gone Breakfast
False
to pool
False
The breakfast
True
lino breakfast
True
modern clean
True
visited pool
True
staff clean
True
wa clean
True
and clean
True
Complimentary breakfast
True
free breakfast
True
Very clean
True
enjoyes breakfast
True
kept clean
False
free breakfast
True
on parking
False
warm breakfast
True
very clean
True
Free parking
True
and breakfast
True
free parking
True
free breakfast
True
and clean
True
are clean
True
microwave coffee
True
is coffee
False
is internet
False
hookup computer
True
hotelHair dryer
True
free coffee
True
day com

have breakfast
False
it breakfast
True
my breakfast
False
about breakfast
False
with breakfast
True
valet parking
True
wa clean
True
are clean
True
mention breakfast
False
hot coffee
True
cold A/C
True
and clean
True
having breakfast
False
are clean
True
and breakfast
True
is parking
False
not parking
True
that parking
True
valet parking
True
and breakfast
True
Valet parking
True
wa clean
True
Valet parking
True
Extremely clean
True
operable Coffee
True
continental breakfast
False
wa clean
True
very clean
True
very clean
True
for breakfast
True
our breakfast
True
croissant coffee
False
find pool
True
instead pool
True
just clean
False
valet parking
True
NoneCleanliness Clean
True
bed clean
False
linen clean
True
Fantastic coffee
True
that breakfast
True
minute breakfast
False
breakfast Parking
False
finding parking
True
Fantastic Clean
True
very clean
True
and coffee
True
and coffee
True
to breakfast
False
and coffee
True
about breakfast
False
about breakfast
False
complimentary breakf

great breakfast
True
valet parking
True
complimentary breakfast
True
wa clean
True
The breakfast
True
Free internet
True
but parking
True
loved breakfast
True
complimentary breakfast
True
loved breakfast
True
Valet parking
True
continental breakfast
False
nice Breakfast
True
and breakfast
True
buffet breakfast
True
area clean
True
complimentary breakfast
True
The breakfast
True
nice clean
True
morning breakfast
False
service laundry
True
provides dryer
True
dryer laundry
True
but clean
True
clean Breakfast
True
great coffee
True
and breakfast
True
average breakfast
True
and breakfast
True
wa pool
True
towel exercise
False
for internet
True
offering coffee
True
an exercise
True
Free breakfast
True
with parking
True
free breakfast
True
free parking
True
dollar parking
False
state parking
True
free parking
True
free parking
True
free parking
True
free parking
True
free parking
True
free parking
True
morning breakfast
False
free parking
True
were clean
False
door Breakfast
False
free break

Small exercise
True
Decent breakfast
True
make breakfast
True
get coffee
False
very clean
True
were clean
False
is coffee
False
very clean
True
very clean
True
were clean
False
for clean
True
no breakfast
False
Morning coffee
True
and clean
True
no breakfast
False
swimming pool
False
no pool
False
very clean
True
or coffee
False
is clean
False
our clean
True
always clean
True
wa clean
True
very clean
True
and clean
True
Morning coffee
True
fridge pool
False
or breakfast
False
were clean
False
wa clean
True
No breakfast
True
breakfast breakfast
False
pastry vending
True
Free coffee
True
so vending
False
option parking
True
No breakfast
True
breakfast breakfast
False
pastry vending
True
Free coffee
True
so vending
False
option parking
True
very clean
True
blow dryer
False
dryer coffee
True
and coffee
True
for clean
True
hair dryer
True
wa clean
True
liked coffee
True
spacious clean
True
wa clean
True
experience Breakfast
True
swimming pool
False
ton pool
False
free breakfast
True
renovat

immaculately clean
True
fridge coffee
False
blow dryer
False
heated pool
False
continental breakfast
False
juice coffee
False
share parking
True
to breakfast
False
very clean
True
your breakfast
True
very clean
True
heated pool
False
wa clean
True
or pool
False
and clean
True
wa parking
True
small pool
True
offer breakfast
True
free breakfast
True
and coffee
True
same parking
True
service breakfast
True
use pool
True
wa parking
True
Complimentary breakfast
True
tea coffee
True
very Breakfast
True
each Pool
True
very clean
True
The breakfast
True
continental breakfast
False
available breakfast
True
and clean
True
what parking
False
any parking
False
very clean
True
out breakfast
False
eating breakfast
True
continental breakfast
False
breakfast Pool
False
and clean
True
issue Breakfast
True
always clean
True
fridge coffee
False
free breakfast
True
re parking
False
bad coffee
False
and clean
True
use pool
True
free breakfast
True
continental breakfast
False
to breakfast
False
hotel parkin

## 3. More

In [131]:
#!pip install negspacy
import spacy
from negspacy.negation import Negex

nlp = spacy.load("en_core_web_sm")
negex = Negex(nlp, ent_types=["NOUN','PRODUCT"])
nlp.add_pipe(negex, last=True)

doc = nlp("Like Carmel, no air-conditioner in rooms but they have a fan for air circulation. Nice pool, nice food, bad car service.")
for e in doc.ents:
    print(e.text, e._.negex)

Carmel False
